<a href="https://colab.research.google.com/github/vicentcamison/idal_ia3/blob/main/5%20Procesado%20del%20lenguaje%20natural/Sesion%203/NLP_08c_Clasificador_binario_LSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis de sentimientos con modelo LSA
Vamos a utilizar un modelo de reducción de la dimensionalidad LSA para clasificar con conjunto de tweets en español

## Carga y preparación de los datos

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)

# Leemos los datos
df = pd.read_csv('tweets_all.csv', index_col=None)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1514 entries, 0 to 1513
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   content   1514 non-null   object
 1   polarity  1514 non-null   object
dtypes: object(2)
memory usage: 23.8+ KB


Tenemos 1514 tweets, de los cuales hay 474 positivos y 637 negativos. El resto son neutros o no tienen polaridad.
Vamos a entrenar sólo con los positivos y negativos

In [ ]:
df = df[(df['polarity']=='P') | (df['polarity']=='N')]

El conjunto no está balanceado pero casi.
Quitamos las columnas que no usamos.

In [ ]:
df.polarity.value_counts()

N    637
P    474
Name: polarity, dtype: int64

## Limpieza de texto
Hacemos un pequeño pre-procesado del texto antes de extraer las características:  
- Quitamos las menciones y las URL del texto porque no aportan valor para el análisis de sentimientos.
- Los hashtag sí que pueden aportar valor así que simplemente quitamos el #.
- Quitamos los signos de puntuación y palabras menores de 3 caracteres.
- Por último quitamos todos los símbolos de puntuación del texto (que forman parte de un token).
- Lematizamos el texto y lo guardamos en otra columna para comparar resultados del clasificador. 

In [ ]:
import re, string, spacy
nlp=spacy.load('es_core_news_md')

pattern2 = re.compile('[{}]'.format(re.escape(string.punctuation))) #elimina símbolos de puntuación

def clean_text(text, lemas=False):
    """Limpiamos las menciones y URL del texto. Luego convertimos en tokens
    y eliminamos signos de puntuación.
    Si lemas=True extraemos el lema, si no dejamos en minúsculas solamente.
    Como salida volvemos a convertir los tokens en cadena de texto"""
    text = re.sub(r'@[\w_]+|https?://[\w_./]+', '', text) #elimina menciones y URL
    tokens = nlp(text)
    tokens = [tok.lemma_.lower() if lemas else tok.lower_ for tok in tokens if not tok.is_punct]
    filtered_tokens = [pattern2.sub('', tok) for tok in tokens] #no quitamos stop-words
    filtered_text = ' '.join(filtered_tokens)
    
    return filtered_text
    

OSError: [E050] Can't find model 'es_core_news_md'. It doesn't seem to be a Python package or a valid path to a data directory.

Aplicamos limpieza a todos los tweets del Dataframe

In [ ]:
df['limpio']=df.content.apply(clean_text)

In [ ]:
#Quitamos tweets vacíos después de la limpieza
df=df[df.content!='']

In [ ]:
df["lemas"]=df.content.apply(clean_text, lemas=True)

In [ ]:
df.head()

### Clasificador
Vamos a usar la librería scikit-learn para aplicar un clasificador sobre la polaridad. Las características de cada Tweet las extraemos en una matriz TF-IDF y luego aplicamos una reducción de dimensiones con un modelo LSA de *topic modeling*.  

Primero dividimos en conjunto de entrenamiento y test.

In [ ]:
from sklearn.model_selection import train_test_split

# Dividmos entre conjunto de entrenamiento y test
# Asignamos un 70% a training y un 30% a test
X_train, X_test, y_train, y_test = train_test_split(df['limpio'], 
                                                    df['polarity'],
                                                    test_size=0.3,
                                                    random_state=0)

In [ ]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

#definimos una función para entrenar y validar cada clasificador
def train_predict_evaluate_model(classifier, 
                                 train_features, train_labels, 
                                 test_features, test_labels):
    # build model    
    classifier.fit(train_features, train_labels)
    # predict using model
    predictions = classifier.predict(test_features) 
    # evaluate model prediction performance   
    print(classification_report(test_labels, predictions))
    return predictions 

# creamos los modelos
modelos = [('Logistic Regression', LogisticRegression()),
           ('Naive Bayes', GaussianNB()),
           ('Linear SVM', SGDClassifier(loss='hinge', max_iter=10000, tol=1e-5)),
           ('RFB VSM', SVC(gamma='scale', C=2))]

### Modelo LSA
Creamos un procesado `pipeline` para calcular la matriz TF-IDF y a partir de esta el modelo LDA

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer

vect = TfidfVectorizer()
svd = TruncatedSVD(n_components=500)

modelo = make_pipeline(vect, svd, Normalizer(copy=False))
#Entrenamos el modelo con el conjunto de train
lsa_train = modelo.fit_transform(X_train)

In [ ]:
lsa_train.shape

In [ ]:
lsa_test = modelo.transform(X_test)
lsa_test.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC

modelos = [('Logistic Regression', LogisticRegression(solver='liblinear')),
           ('Naive Bayes', GaussianNB()),
           ('Linear SVM', SGDClassifier(loss='hinge', max_iter=10000, tol=1e-5)),
           ('RFB SVM', SVC(gamma='scale', C=2))]

for m, clf in modelos:
    print('Modelo {} con características LSA 500 dims'.format(m))
    #entrenamos sobre train
    clf.fit(lsa_train, y_train)
    # Predecimos sobre el conjunto de test
    prediccion = clf.predict(lsa_test)
    print(classification_report(y_test, prediccion))


Una vez establecido el modelo de clasificación a utilizar podemos integrar todos los paso en el `pipeline`:

In [ ]:
vect = TfidfVectorizer()
svd = TruncatedSVD(n_components=500)
modelSVM = SVC(gamma='scale', C=2)

modelo = make_pipeline(vect, svd, Normalizer(copy=False), modelSVM)
#Entrenamos el modelo con el conjunto de train
modelo.fit(X_train, y_train)

In [ ]:
prediccion = modelo.predict(X_test)
print(classification_report(y_test, prediccion))